In [1]:
from sagemaker import Session
session = Session()
# replace with your bucket and prefix
bucket = 'studentperformancebucket' 
prefix = 'clarify'
region = session.boto_region_name
# Define IAM role
from sagemaker import get_execution_role
import pandas as pd
import numpy as np
import urllib
import os
import boto3
import matplotlib.pyplot as plt
import seaborn as sns
from random import randrange, choice
from sklearn.neighbors import NearestNeighbors
role = get_execution_role()
role = get_execution_role()

In [2]:
data_bucket = 'studentperformancebucket' # replace with your bucket name
key = 'clarify/port modified.csv' # replace with your object key

s3 = boto3.resource('s3')
s3.Bucket(data_bucket).download_file(key, 'port.csv')

data = pd.read_csv('./port.csv')

pd.set_option('display.max_rows', 20) 
data

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,MS,F,19,R,GT3,T,2,3,services,other,...,5,4,2,1,2,5,4,10,11,10
645,MS,F,18,U,LE3,T,3,1,teacher,services,...,4,3,4,1,1,1,4,15,15,16
646,MS,F,18,U,GT3,T,1,1,other,other,...,1,1,1,1,1,5,6,11,12,9
647,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,6,10,10,10


In [3]:
def SMOTE(T, N, k):

    n_minority_samples, n_features = T.shape

    if N < 100:
       N = 100
       pass

    if (N % 100) != 0:
       raise ValueError("N must be < 100 or multiple of 100")

    N = N/100
    n_synthetic_samples = N * n_minority_samples
    n_synthetic_samples = int(n_synthetic_samples)
    n_features = int(n_features)
    S = np.zeros(shape=(n_synthetic_samples, n_features))

    neigh = NearestNeighbors(n_neighbors = k)
    neigh.fit(T)

    #Calculate synthetic samples
    for i in range(n_minority_samples):
       nn = neigh.kneighbors(T[i], return_distance=False)
       for n in range(N):
          nn_index = choice(nn[0])
          while nn_index == i:
             nn_index = choice(nn[0])

          dif = T[nn_index] - T[i]
          gap = np.random.random()
          S[n + i * N, :] = T[i,:] + gap * dif[:]

    return S

columns = data.columns.to_numpy()
iteration_count = 30
new_df = pd.DataFrame(columns=columns)

for i in range(iteration_count):
    for k in data.iterrows():
        data_obj = {}
        for j in range(columns.size):
            random_index = np.random.randint(0,13, dtype='int')
            data_obj[columns[j]] = data.loc[random_index][columns[j]]
        new_df = new_df.append(data_obj, ignore_index=True)

data = data.append(new_df, ignore_index=True)

In [6]:
data.to_csv('synthetic_data_port.csv', index = False)

In [7]:
from sagemaker.s3 import S3Uploader

synthetic_data = S3Uploader.upload('synthetic_data_port.csv', 's3://{}/{}'.format(bucket, prefix))